In [34]:
import numpy as np
import pandas as pd

In [35]:
class binary():
    def __init__(self,capas=1,nodos=1):
        self.capas = capas
        self.nodos = nodos
        self.forma = (self.capas,self.nodos)
    def inicializar(self):
        self.red = []
        for capa in range(self.capas):
            w = np.random.normal(size = (self.nodos,self.nodos))
            b = np.random.normal(size = self.nodos)
            self.red.append([w,b])
        w_f = np.random.normal(size = self.nodos)
        b_f = np.random.normal(size = 1)
        self.red.append([w_f,b_f])
    def mutar(self,alfa=0.1):
        red = []
        for r in self.red:
            r0 = r[0]+alfa*np.random.normal(size = r[0].shape)
            r1 = r[1]+alfa*np.random.normal(size = r[1].shape)
            red.append([r0,r1])
        hijo = binary(self.capas,self.nodos)
        hijo.inicializar()
        hijo.red = red
        return hijo
    def cruzar(self,other):
        if self.forma == other.forma:
            hijo = binary(self.capas,self.nodos)
            hijo.inicializar()
            red = []
            for i in range(len(self.red)):
                r0 = (self.red[i][0] + other.red[i][0])/2.0
                r1 = (self.red[i][1] + other.red[i][1])/2.0
                red.append([r0,r1])
            hijo.red = red
            return hijo
        else:
            return None
    def clonar(self):
        hijo = binary(self.capas,self.nodos)
        hijo.inicializar()
        red = []
        for i in range(len(self.red)):
            r0 = self.red[i][0]
            r1 = self.red[i][1]
            red.append([r0,r1])
        hijo.red = red
        return hijo
    def sumar(self,other):
        for i in range(len(self.red)):
            self.red[i][0] += other.red[i][0]
            self.red[i][1] += other.red[i][1]
    def normalizar(self,n_pasos):
        for i in range(len(self.red)):
            self.red[i][0] /= n_pasos
            self.red[i][1] /= n_pasos
    def distancia(self,other):
        if self.forma == other.forma:
            distancia = 0
            for i in range(len(self.red)):
                r0 = (self.red[i][0] - other.red[i][0])**2
                r1 = (self.red[i][1] - other.red[i][1])**2
                distancia = distancia + r0.sum().sum()+r1.sum().sum()
                distancia = np.sqrt(distancia)
            return distancia
        else:
            return None
    def norma(self):
        if True:
            distancia = 0
            for i in range(len(self.red)):
                r0 = (self.red[i][0])**2
                r1 = (self.red[i][1])**2
                distancia = distancia + r0.sum().sum()+r1.sum().sum()
                distancia = np.sqrt(distancia)
            return distancia
        else:
            return None
        
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def predecir(self,entrada):
        for r in self.red:
            entrada = np.dot(entrada,r[0])
            entrada = entrada + r[1]
            entrada = self.sigmoid(entrada)
        entrada = entrada.round()
        entrada = int(entrada)
        return entrada
    def evaluar(self,muestra):
        target = muestra.target
        u = muestra.columns.tolist()
        v = []
        for i in u:
            if i != 'target':
                v.append(i)
        entrada = muestra.loc[:,v].values.tolist()
        resultado = target.tolist()
        suma = 0
        cuenta = 0
        for i in range(len(entrada)):
            u = self.predecir(entrada[i])
            v = resultado[i]
            if u == v:
                suma += 1
            cuenta += 1
        u = float(suma)/float(cuenta)
        return u  
    def razon(self,other,n_pasos):
        if self.forma == other.forma:
            hijo = binary(self.capas,self.nodos)
            hijo.inicializar()
            red = []
            for i in range(len(self.red)):
                r0 = (-self.red[i][0] + other.red[i][0])/(n_pasos)+self.red[i][0]
                r1 = (-self.red[i][1] + other.red[i][1])/(n_pasos)+self.red[i][1]
                red.append([r0,r1])
            self.red = red
                
            

In [36]:
def crear_traza(individuo,n_pasos,muestra,n_sample,alfa=0.1):
    ''' Devuelve el mejor individuo, el individuo promedio y el ranking
    tanto del mejor individuo como del promedio'''
    prediccion_traza = 0
    mejor_individuo = individuo.clonar()
    mejor_prediccion = mejor_individuo.evaluar(muestra.sample(n_sample))
    trazas = []
    for i in range(n_pasos):
        hijo = individuo.mutar(alfa)
        t = hijo.clonar()
        trazas.append(t)
        prediccion = hijo.evaluar(muestra.sample(n_sample))
        if prediccion > mejor_prediccion:
            mejor_prediccion = prediccion
            mejor_individuo = hijo.clonar()
        prediccion_traza += prediccion
        individuo = hijo.clonar()
    traza = promediar(trazas)
    red = []
    for r in range(len(individuo.red)):
        r0 = individuo.red[r][0]+(traza.red[r][0]-individuo.red[r][0])/np.sqrt(len(trazas))
        r1 = individuo.red[r][1]+(traza.red[r][1]-individuo.red[r][1])/np.sqrt(len(trazas))
        red.append([r0,r1])
    traza.red = red
    prediccion_traza = prediccion_traza/float(n_pasos)
    return mejor_prediccion, mejor_individuo, prediccion_traza, traza

In [37]:
def promediar(individuos):
    individuo_0 = individuos[0].clonar()
    red = individuo_0.red[:]
    for r in range(len(red)):
        r0 = red[r][0]
        r1 = red[r][1]
        red[r][0]= -r0+red[r][0]
        red[r][1]= -r1+red[r][1]
        
    for individuo in individuos:
        for r in range(len(red)):
            r0 = (individuo.red[r][0][:])
            r1 = (individuo.red[r][1][:])
            red[r][0]=r0+red[r][0]
            red[r][1]=r1+red[r][1]
    n_red = []
    for r in range(len(red)):
        red[r][0]=red[r][0]/float(len(individuos))
        red[r][1]=red[r][1]/float(len(individuos))
        n_red.append([red[r][0],red[r][1]])
    individuo_0.red = n_red
    individuo_0.red = red
    return individuo_0

In [38]:
titanic = pd.read_csv('train.csv')
titanic.groupby('Embarked').size()
titanic = pd.read_csv('train.csv')
titanic = titanic[['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
titanic.loc[titanic.Embarked=='C','Embarked']=-1
titanic.loc[titanic.Embarked=='Q','Embarked']=0
titanic.loc[titanic.Embarked=='S','Embarked']=1
titanic['target'] = titanic.Survived
titanic['Pclass_1']=np.where(titanic.Pclass==1,1,0)
titanic['Pclass_2']=np.where(titanic.Pclass==2,1,0)
titanic['Pclass_3']=np.where(titanic.Pclass==3,1,0)
titanic['Sex']=np.where(titanic.Sex=='male',1,0)
titanic = titanic.fillna(titanic.mean())
titanic[['Age']] = titanic[['Age']]/titanic[['Age']].mean()
titanic[['Fare']] = titanic[['Fare']]/titanic[['Fare']].mean()
del titanic['Pclass'],titanic['Survived']


In [39]:
titanic_train = titanic[titanic.target==0].sample(250)\
                .append(titanic[titanic.target==1].sample(250))
titanic_test = titanic.loc[~titanic.index.isin(titanic_train.index),:]

In [ ]:
individuo = binary(0,9)
individuo.inicializar()

In [ ]:
mejor_prediccion = 0
n= 1000
alfa = 1
alpha = alfa*np.exp(np.log(10**-5)/n)
for i in range(n):
    mejor_tprediccion = 0
    alfa = alpha ** i
    print(alfa)
    for i in range(10):
        mp,mi,pt,t = crear_traza(individuo,25,titanic_train,200,alfa = alfa)
        if mp > mejor_prediccion:
            mejor_prediccion = mp
            mejor_individuo = mi.clonar()
            
        if mejor_tprediccion < pt:
            mejor_tprediccion = pt
            mejor_traza = t.clonar()
    print('mejor_prediccion',mejor_prediccion,'mejor_tprediccion',mejor_tprediccion,
         'sobre test',mejor_traza.evaluar(titanic_test))
    print(individuo.norma())
    individuo = mejor_traza.clonar()

1.0
mejor_prediccion 0.805 mejor_tprediccion 0.6788 sobre test 0.6112531969309463
2.992789663892142
0.9885530946569389
mejor_prediccion 0.805 mejor_tprediccion 0.7316000000000001 sobre test 0.710997442455243
11.927802956764062
0.9772372209558107
mejor_prediccion 0.805 mejor_tprediccion 0.7282 sobre test 0.7468030690537084
17.51759137464483
0.9660508789898135
mejor_prediccion 0.83 mejor_tprediccion 0.7810000000000001 sobre test 0.7135549872122762
21.637598477718715
0.954992586021436
mejor_prediccion 0.83 mejor_tprediccion 0.7726 sobre test 0.7391304347826086
27.65266629014849
0.9440608762859235
mejor_prediccion 0.84 mejor_tprediccion 0.7772 sobre test 0.6828644501278772
31.50219275289916
0.9332543007969911
mejor_prediccion 0.84 mejor_tprediccion 0.7679999999999998 sobre test 0.7800511508951407
30.857401092632013
0.9225714271547633
mejor_prediccion 0.84 mejor_tprediccion 0.7710000000000002 sobre test 0.7647058823529411
28.95007393308946
0.9120108393559099
mejor_prediccion 0.84 mejor_tpre

mejor_prediccion 0.88 mejor_tprediccion 0.8030000000000002 sobre test 0.7340153452685422
110.39817663369352
0.45185594437492294
mejor_prediccion 0.88 mejor_tprediccion 0.8018 sobre test 0.7340153452685422
112.51224164949066
0.44668359215096365
mejor_prediccion 0.88 mejor_tprediccion 0.8072 sobre test 0.731457800511509
114.19341746791588
0.4415704473533131
mejor_prediccion 0.88 mejor_tprediccion 0.8001999999999999 sobre test 0.7391304347826086
114.56996211485249
0.43651583224016655
mejor_prediccion 0.88 mejor_tprediccion 0.8086000000000001 sobre test 0.7570332480818415
110.26170976594325
0.4315190768277658
mejor_prediccion 0.88 mejor_tprediccion 0.8054000000000001 sobre test 0.7391304347826086
107.98202914840165
0.4265795188015932
mejor_prediccion 0.88 mejor_tprediccion 0.8042000000000001 sobre test 0.7519181585677749
106.72813001152198
0.4216965034285828
mejor_prediccion 0.88 mejor_tprediccion 0.8060000000000003 sobre test 0.7442455242966752
106.27065856760112
0.416869383470336
mejor_p

mejor_prediccion 0.895 mejor_tprediccion 0.8016000000000001 sobre test 0.7365728900255755
108.87213019212304
0.21134890398366518
mejor_prediccion 0.895 mejor_tprediccion 0.8066000000000001 sobre test 0.7416879795396419
109.03049906819724
0.20892961308540445
mejor_prediccion 0.895 mejor_tprediccion 0.8021999999999998 sobre test 0.7340153452685422
109.90458732105893
0.20653801558105345
mejor_prediccion 0.895 mejor_tprediccion 0.7991999999999999 sobre test 0.7340153452685422
108.59941593798351
0.20417379446695344
mejor_prediccion 0.895 mejor_tprediccion 0.805 sobre test 0.7391304347826086
109.78908161851085
0.2018366363681566
mejor_prediccion 0.895 mejor_tprediccion 0.7998000000000001 sobre test 0.7365728900255755
110.35151237060518
0.19952623149688845
mejor_prediccion 0.895 mejor_tprediccion 0.8016 sobre test 0.7493606138107417
109.33972925789942
0.19724227361148586
mejor_prediccion 0.895 mejor_tprediccion 0.8050000000000002 sobre test 0.7442455242966752
109.60101104119099
0.194984459975

In [ ]:
best_model = mejor_traza


In [ ]:
best_model.evaluar(titanic_test)

In [ ]:
titanic = pd.read_csv('test.csv')
titanic = titanic[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
titanic.loc[titanic.Embarked=='C','Embarked']=-1
titanic.loc[titanic.Embarked=='Q','Embarked']=0
titanic.loc[titanic.Embarked=='S','Embarked']=1
titanic['Pclass_1']=np.where(titanic.Pclass==1,1,0)
titanic['Pclass_2']=np.where(titanic.Pclass==2,1,0)
titanic['Pclass_3']=np.where(titanic.Pclass==3,1,0)
titanic['Sex']=np.where(titanic.Sex=='male',1,0)
titanic = titanic.fillna(titanic.mean())
titanic[['Age']] = titanic[['Age']]/titanic[['Age']].mean()
titanic[['Fare']] = titanic[['Fare']]/titanic[['Fare']].mean()
del titanic['Pclass']

In [ ]:
scores = []
for index, row in titanic.iterrows():
    scores.append(best_model.predecir(row.values))


In [ ]:
resultado = pd.read_csv('test.csv')
resultado['Survived'] = scores
resultado.Survived = resultado.Survived.round().astype(int)
resultado[['PassengerId','Survived']].to_csv('result.csv',index=False)